In [12]:
import random
import json
import helper
import models
import numpy as np
import pandas as pd
import importlib as imp
from datetime import datetime
from matplotlib import pyplot as plt
from __future__ import print_function

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision import datasets, transforms, utils

rs = 2018
random.seed(rs)
%matplotlib inline

## Get the tensor data

In [2]:
use_cuda = torch.cuda.is_available()
print('Using cuda: {}'.format(use_cuda))
torch.manual_seed(rs)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
batch_size = 1000
n_layers = 1

Using cuda: False


In [3]:
train_loader = torch.utils.data.DataLoader(
    datasets.EMNIST('data/', train=True, split='digits', download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.EMNIST('data/', train=False, split='digits', transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader_ordered = torch.utils.data.DataLoader(
    datasets.EMNIST('data/', train=False, split='digits', transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=len(test_loader.dataset), shuffle=False, **kwargs)

## Testing across all parameters

**Note:** Netx represents an x layer network between 1 and 4 layers

In [9]:
#Parameter Tuning
epochs= [5,10,15]
batch_size= [100,200,300]
dropouts = [.2, .5, .75]
layers=[1,2,3,4]

score=[]
data_dict={}
key= 0 
for batch in batch_size:
    train_loader = torch.utils.data.DataLoader(
        datasets.EMNIST('data/', train=True, split='digits', download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch, shuffle=True, **kwargs)

    test_loader = torch.utils.data.DataLoader(
        datasets.EMNIST('data/', train=False, split='digits', transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch, shuffle=True, **kwargs)

    test_loader_ordered = torch.utils.data.DataLoader(
        datasets.EMNIST('data/', train=False, split='digits', transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=len(test_loader.dataset), shuffle=False, **kwargs)
    for layer in layers:
        for drops in dropouts:
            model = models.get_model(n_layers, dropout_rate=drops, n_filters=10, filter_size=5, fc_units=50)

            optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.5)
            start = datetime.now() 
            for epoch in epochs:
                models.train(model, optimizer, train_loader, use_cuda, device, epoch, layers=layer)
                print('Time elapsed {}'.format(datetime.now() - start))
                test_loss, correct = models.test(model, optimizer, test_loader, use_cuda, device, epoch, layers=n_layers)
                scores = test_loss
                print(epoch, batch, drops)

                #Save Iterations in Dictionary
                data_dict[key]={'layer':layer, 'epoch':epoch, 'batch_size':batch, 'dropout_rate':drops}
                score.append(scores)
                key += 1  #update Dictionary key

data= list(zip(score,data_dict.values()))
with open('results/pytorch_results.json','w') as outfile:
    json.dump(data,outfile)

Train Epoch: 5 [0/240000 (0%)]	Loss: 2.326792
Train Epoch: 5 [1000/240000 (0%)]	Loss: 2.298206
Train Epoch: 5 [2000/240000 (1%)]	Loss: 2.275814
Train Epoch: 5 [3000/240000 (1%)]	Loss: 2.263617
Train Epoch: 5 [4000/240000 (2%)]	Loss: 2.246646
Train Epoch: 5 [5000/240000 (2%)]	Loss: 2.230634
Train Epoch: 5 [6000/240000 (2%)]	Loss: 2.258646
Train Epoch: 5 [7000/240000 (3%)]	Loss: 2.185967
Train Epoch: 5 [8000/240000 (3%)]	Loss: 2.154522
Train Epoch: 5 [9000/240000 (4%)]	Loss: 2.115901
Train Epoch: 5 [10000/240000 (4%)]	Loss: 2.098731
Train Epoch: 5 [11000/240000 (5%)]	Loss: 2.106208
Train Epoch: 5 [12000/240000 (5%)]	Loss: 2.059592
Train Epoch: 5 [13000/240000 (5%)]	Loss: 2.056937
Train Epoch: 5 [14000/240000 (6%)]	Loss: 2.051334
Train Epoch: 5 [15000/240000 (6%)]	Loss: 1.980890
Train Epoch: 5 [16000/240000 (7%)]	Loss: 1.936581
Train Epoch: 5 [17000/240000 (7%)]	Loss: 1.912089
Train Epoch: 5 [18000/240000 (8%)]	Loss: 1.878988
Train Epoch: 5 [19000/240000 (8%)]	Loss: 1.796971
Train Epoch: 

Train Epoch: 5 [161000/240000 (67%)]	Loss: 0.239279
Train Epoch: 5 [162000/240000 (68%)]	Loss: 0.389505
Train Epoch: 5 [163000/240000 (68%)]	Loss: 0.399636
Train Epoch: 5 [164000/240000 (68%)]	Loss: 0.524110
Train Epoch: 5 [165000/240000 (69%)]	Loss: 0.382741
Train Epoch: 5 [166000/240000 (69%)]	Loss: 0.209045
Train Epoch: 5 [167000/240000 (70%)]	Loss: 0.466427
Train Epoch: 5 [168000/240000 (70%)]	Loss: 0.349516
Train Epoch: 5 [169000/240000 (70%)]	Loss: 0.363623
Train Epoch: 5 [170000/240000 (71%)]	Loss: 0.363059
Train Epoch: 5 [171000/240000 (71%)]	Loss: 0.296950
Train Epoch: 5 [172000/240000 (72%)]	Loss: 0.507606
Train Epoch: 5 [173000/240000 (72%)]	Loss: 0.331676
Train Epoch: 5 [174000/240000 (72%)]	Loss: 0.352191
Train Epoch: 5 [175000/240000 (73%)]	Loss: 0.370896
Train Epoch: 5 [176000/240000 (73%)]	Loss: 0.430103
Train Epoch: 5 [177000/240000 (74%)]	Loss: 0.428244
Train Epoch: 5 [178000/240000 (74%)]	Loss: 0.295272
Train Epoch: 5 [179000/240000 (75%)]	Loss: 0.308679
Train Epoch:

Train Epoch: 5 [79000/240000 (33%)]	Loss: 0.762474
Train Epoch: 5 [80000/240000 (33%)]	Loss: 0.796042
Train Epoch: 5 [81000/240000 (34%)]	Loss: 0.897549
Train Epoch: 5 [82000/240000 (34%)]	Loss: 0.806355
Train Epoch: 5 [83000/240000 (35%)]	Loss: 0.751665
Train Epoch: 5 [84000/240000 (35%)]	Loss: 0.791398
Train Epoch: 5 [85000/240000 (35%)]	Loss: 0.805529
Train Epoch: 5 [86000/240000 (36%)]	Loss: 0.670129
Train Epoch: 5 [87000/240000 (36%)]	Loss: 0.679174
Train Epoch: 5 [88000/240000 (37%)]	Loss: 0.821832
Train Epoch: 5 [89000/240000 (37%)]	Loss: 0.795781
Train Epoch: 5 [90000/240000 (38%)]	Loss: 0.633927
Train Epoch: 5 [91000/240000 (38%)]	Loss: 0.849702
Train Epoch: 5 [92000/240000 (38%)]	Loss: 0.714790
Train Epoch: 5 [93000/240000 (39%)]	Loss: 0.733404
Train Epoch: 5 [94000/240000 (39%)]	Loss: 0.766937
Train Epoch: 5 [95000/240000 (40%)]	Loss: 0.697761
Train Epoch: 5 [96000/240000 (40%)]	Loss: 0.699044
Train Epoch: 5 [97000/240000 (40%)]	Loss: 0.680159
Train Epoch: 5 [98000/240000 (4

Train Epoch: 5 [237000/240000 (99%)]	Loss: 0.498106
Train Epoch: 5 [238000/240000 (99%)]	Loss: 0.395772
Train Epoch: 5 [239000/240000 (100%)]	Loss: 0.367916
Time elapsed 0:00:54.789812

Test set: Average loss: 0.2812, Accuracy: 36821.0/40000 (92%)

5 100 0.5


FileNotFoundError: [Errno 2] No such file or directory: '../results/pytorch_results.json'